# Auto-Batched Joint Distributions: A Gentle Tutorial

##### Copyright 2020 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/probability/examples/Modeling_with_JointDistribution"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
  </td>
  <td><a target="_blank" href="https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/probability/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

### はじめに

TensorFlow Probability (TFP) は、ユーザーが確率的グラフィカルモデルを数学的な形式で簡単に表現できるようにすることで、確率的推論を容易にする多数の `JointDistribution` 抽象化を提供します。抽象化により、モデルからサンプリングし、モデルからのサンプルの対数確率を評価するためのメソッドが生成されます。このチュートリアルでは、元の `JointDistribution` 抽象化の後に開発された「自動バッチ処理」バリアントを見ていきます。自動バッチ処理されていない元の抽象化と比較して、自動バッチ処理されたバージョンは使いやすく人間工学的であるため、多くのモデルをより少ないボイラープレートで表現できます。このコラボでは、単純なモデルを詳細に調査し、自動バッチ処理が解決する問題を明らかにし、TFP 形状の概念について詳しく説明します。

自動バッチ処理が導入される前は、確率モデルを表現するためのさまざまな構文スタイルに対応する `JointDistribution` のいくつかの異なるバリアントがありました (`JointDistributionSequential`、`JointDistributionNamed`、`JointDistributionCoroutine`など)。自動バッチ処理では、これらすべての `AutoBatched` バリアントを利用できます。このチュートリアルでは、`JointDistributionSequential` と `JointDistributionSequentialAutoBatched` の違いを見ていきますが、ここで行うことはすべて、基本的に変更せずに他のバリアントに適用できます。


### 依存関係と前提条件


In [ ]:
#@title Import and set ups{ display-mode: "form" }

import functools
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp

tfd = tfp.distributions

### 前提条件: ベイズ回帰問題

非常に単純なベイズ回帰シナリオを検討します。

$$ \begin{align*} m &amp; \sim \text{Normal}(0, 1) \ b &amp; \sim \text{Normal}(0, 1) \ Y &amp; \sim \text{Normal}(mX + b, 1) \end{align*} $$

このモデルでは、`m` および `b` は標準正規分布から抽出されます。観測値 `Y` は、平均が確率変数 `m` および `b`、および、いくつかの (非ランダム、既知の) 共変量 `X` に依存する正規分布から抽出されます。(簡単にするために、この例では、すべての確率変数のスケールが既知であると想定します。)

このモデルで推論を実行するには、共変量 `X` と観測値 `Y` の両方を知る必要があります。ただし、このチュートリアルでは、`X` のみが必要なので、単純なダミー `X` を定義します。

In [ ]:
X = np.arange(7)
X

array([0, 1, 2, 3, 4, 5, 6])

### デシデラタ

確率的推論では、多くの場合、以下の 2 つの基本的な演算を実行します。

- `sample`: モデルからサンプルを抽出する
- `log_prob`: モデルからのサンプルの対数確率を計算します。

TFP の `JointDistribution` 抽象化の主な利点 (および確率的プログラミングへの他の多くのアプローチ) として、ユーザーはモデルを*一回*を記述すると、`sample` および `log_prob` の両方の計算を実行できます。

データセットに 7 つの点 (`X.shape = (7,)`) があることに注意して、`JointDistribution` のデシデラタを述べます。

- `sample()` は、スカラー勾配、スカラーバイアス、およびベクトル観測値にそれぞれ対応する、形状 `[(), (), (7,)`] を持つ `Tensors` のリストを生成する必要があります。
- `log_prob(sample())` はスカラーを生成する必要があります (特定の勾配、バイアス、および観測値の対数確率)。
- `sample([5, 3])` はモデルからのサンプルの `(5, 3)`-*バッチ*を表す形状が`[(5, 3), (5, 3), (5, 3, 7)]`の `Tensors` のリストを生成する必要があります。
- `log_prob(sample([5, 3]))` は形状 (5, 3) の `Tensor` を生成する必要があります。

次に、一連の `JointDistribution` モデルを見て、上記の目標を達成する方法を確認しながら TFP の形状についても見ていきます。

ネタバレ注意: ボイラープレートを追加せずに上記のデシデラタを満たすには[自動バッチ処理](#scrollTo=_h7sJ2bkfOS7)を使用します。 

### 最初の試み、`JointDistributionSequential`

In [ ]:
jds = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

これは、モデルをコードに直接変換したものです。勾配 `m` とバイアス `b` は単純です。`Y` は、`lambda` 関数を使用して定義されます。一般的なパターンは、`JointDistributionSequential` (JDS) の $k$ の `lambda` 関数がモデル内の事前の $k$ 分布を使用することです。「逆」の順序に注意してください。

`sample_distributions` を呼び出します。これは、サンプル*と*サンプルの生成に使用された基礎となる「サブディストリビューション」を返します。（`sample` を呼び出すことでサンプルだけを作成することもできます。分布はチュートリアルの後半で使用するので、用意しておくと便利です。) 生成されたサンプルには問題ありません。

In [ ]:
dists, sample = jds.sample_distributions()
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

ただし、`log_prob` は、望ましくない形状の結果を生成します。

In [ ]:
jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-4.4777603, -4.6775575, -4.7430477, -4.647725 , -4.5746684,
       -4.4368567, -4.480562 ], dtype=float32)>

また、複数のサンプリングは機能しません。

In [ ]:
try:
  jds.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


問題がどこにあるか見てみましょう。

### 簡単な見直し: バッチとイベントの形状

TFP では、通常の (`JointDistribution` ではない) 確率分布には&lt;em data-md-type="emphasis"&gt;イベント形状&lt;/em&gt;と&lt;em data-md-type="emphasis"&gt;バッチ形状&lt;/em&gt;があります。TFP を効果的に使用するには、これらの違いを理解することが重要です。

- イベントの形状は、分布からの 1 つの抽出の形状を表します。抽出は次元間で依存する場合があります。スカラー分布の場合、イベントの形状は [] です。5 次元の MultivariateNormal の場合、イベントの形状は [5] です。
- バッチ形状は、独立した、同一に分散されていない抽出である「バッチ」の分布を表します。単一の Python オブジェクトで分布のバッチを表すことは、TFP が大規模な効率を達成するための重要な方法の 1 つです。

ここでは、分布からの単一のサンプルで `log_prob` を呼び出す場合、結果は常に*バッチ*の形状と一致する (つまり、右端の次元を持つ) 形状になります。

形状の詳細については、[「TensorFlow 分布の形状について」のチュートリアル](https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes)を参照してください。


### `log_prob(sample())` がスカラーを生成しない理由 

バッチとイベントの形状に関する知識を使用して、`log_prob(sample())` で何が起こっているかを調べてみましょう。サンプルは以下のとおりです。

In [ ]:
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

分布は以下のとおりです。

In [ ]:
dists

[<tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>]

対数確率は、部分の (一致した) 要素での劣確率分布の対数確率を合計することによって計算されます。

In [ ]:
log_prob_parts = [dist.log_prob(s) for (dist, s) in zip(dists, sample)]
log_prob_parts

[<tf.Tensor: shape=(), dtype=float32, numpy=-2.3113134>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.1357536>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([-1.0306933, -1.2304904, -1.2959809, -1.200658 , -1.1276014,
        -0.9897899, -1.0334952], dtype=float32)>]

In [ ]:
np.sum(log_prob_parts) - jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

したがって、`log_prob_parts` の 3 番目のサブコンポーネントが 7 テンソルであるため、対数確率計算が 7 テンソルを返すと説明できます。しかし、なぜでしょうか？

数学の定式化で `Y` の分布に対応する `dists` の最後の要素には、`[7]` の `batch_shape` があることがわかります。言い換えると、`Y` での分布は、7 つの独立した法線のバッチです (平均が異なり、この場合は同じスケールです)。

問題が何だかわかりました。JDS では、`Y` の分布には `batch_shape=[7]` があります。JDS のサンプルは、`m` と `b` のスカラーと、7 つの独立した法線の「バッチ」を表しています。`log_prob` は、7 つの別々の対数確率を計算します。それぞれが `m` と `b`を抽出する対数確率、そして、`X[i]` での単一の観測 `Y[i]` を表しています。

### `log_prob(sample())` を `Independent` で修正する

`dists[2]` には`event_shape=[]` と `batch_shape=[7]` があることを思い出してください。

In [ ]:
dists[2]

<tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>

バッチの次元をイベントの次元に変換する TFP の `Independent` メタ分布を使用することにより、これを `event_shape=[7]` と `batch_shape=[]` の分布に変換できます。(`Y` の分布であり、`_i` が `Independent` ラッピングの代わりになるため、名前を`y_dist_i` に変更します。) 

In [ ]:
y_dist_i = tfd.Independent(dists[2], reinterpreted_batch_ndims=1)
y_dist_i

<tfp.distributions.Independent 'IndependentJointDistributionSequential_sample_distributions_Normal' batch_shape=[] event_shape=[7] dtype=float32>

これで、7 ベクトルの `log_prob` はスカラーになります。

In [ ]:
y_dist_i.log_prob(sample[2])

<tf.Tensor: shape=(), dtype=float32, numpy=-7.9087086>

裏で、`Independent` はバッチ全体の和を計算します。

In [ ]:
y_dist_i.log_prob(sample[2]) - tf.reduce_sum(dists[2].log_prob(sample[2]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

実際、これを使用して新しい `jds_i` を作成できます (繰り返しますが、`i` は `Independent` を表します)。ここで、`log_prob` はスカラーを返します。

In [ ]:
jds_i = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m*X + b, scale=1.),
        reinterpreted_batch_ndims=1)
])

jds_i.log_prob(sample)

<tf.Tensor: shape=(), dtype=float32, numpy=-11.355776>

注意事項:

- `jds_i.log_prob(s)` は `tf.reduce_sum(jds.log_prob(s))` と*同じではありません*。前者は、同時分布の「正しい」対数確率を生成します。後者は 7 テンソルの合計であり、その各要素は `m`、`b` の対数確率、および対数確率 `Y` の単一要素の合計です。したがって、`m` と `b` がオーバーカウントされます。(`log_prob(m) + log_prob(b) + log_prob(Y)` では、TFP は TF および NumPy のブロードキャストルール (ベクトルにスカラーを追加すると、ベクトルサイズの結果が生成される) に従うため、例外をスローせずに結果を返します。)
- この特定のケースでは、`Independent(Normal(...))`の代わりに `MultivariateNormalDiag` を使用して、問題を解決し、同じ結果を達成できます。`MultivariateNormalDiag` はベクトル値分布です（つまり、すでにベクトルイベント形状を持っています）。確かに `MultivariateNormalDiag` は、`Independent` と `Normal` を合わせて実装できます。ベクトル`V`が与えられた場合、`n1 = Normal(loc=V)` および `n2 = MultivariateNormalDiag(loc=V)` からのサンプルは区別できません。これらの分布の違いは、`n1.log_prob(n1.sample())` がベクトルであり、`n2.log_prob(n2.sample())` がスカラーであることです。

### 複数のサンプル

複数のサンプリングは機能しません。

In [ ]:
try:
  jds_i.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


理由を考えてみましょう。`jds_i.sample([5, 3])` を呼び出すと、最初に`m` と `b` のサンプルを抽出します。それぞれの形状は `(5, 3)` です。次に、次の方法で `Normal` 分布を構築します。

```
tfd.Normal(loc=m*X + b, scale=1.)
```

ただし、`m` の形状が `(5, 3)` で、`X` の形状が `7` の場合、それらを乗算することはできません。そのためにエラーが発生します。

In [ ]:
m = tfd.Normal(0., 1.).sample([5, 3])
try:
  m * X
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


この問題を解決するために、`Y` の分布に必要なプロパティについて考えてみましょう。`jds_i.sample([5, 3])` を呼び出した場合、`m` と `b` の両方の形状が `(5, 3)` になります。`Y` 分布で `sample` を呼び出すと、どのような形状になるでしょうか？ 明らかに `(5, 3, 7)` です。バッチポイントごとに、`X` と同じサイズのサンプルが必要です。TensorFlow のブロードキャスト機能を使用すると、次のように次元を追加できます。

In [ ]:
m[..., tf.newaxis].shape

TensorShape([5, 3, 1])

In [ ]:
(m[..., tf.newaxis] * X).shape

TensorShape([5, 3, 7])

`m` と `b` の両方に軸を追加すると、複数のサンプルをサポートする新しい JDS を定義できます。

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

shaped_sample = jds_ia.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-1.1133379 ,  0.16390413, -0.24177533],
        [-1.1312429 , -0.6224666 , -1.8182136 ],
        [-0.31343174, -0.32932565,  0.5164407 ],
        [-0.0119963 , -0.9079621 ,  2.3655841 ],
        [-0.26293617,  0.8229698 ,  0.31098196]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.02876974,  1.0872147 ,  1.0138507 ],
        [ 0.27367726, -1.331534  , -0.09084719],
        [ 1.3349475 , -0.68765205,  1.680652  ],
        [ 0.75436825,  1.3050154 , -0.9415123 ],
        [-1.2502679 , -0.25730947,  0.74611956]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 7), dtype=float32, numpy=
 array([[[-1.8258233e+00, -3.0641669e-01, -2.7595463e+00, -1.6952467e+00,
          -4.8197951e+00, -5.2986512e+00, -6.6931367e+00],
         [ 3.6438566e-01,  1.0067395e+00,  1.4542470e+00,  8.1155670e-01,
           1.8868095e+00,  2.3877139e+00,  1.0195159e+00],
         [-8.3624744e-01,  1.2518480e+00,  1.0943471e+00, 

In [ ]:
jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.483114 , -10.139662 , -11.514159 ],
       [-11.656767 , -17.201958 , -12.132455 ],
       [-17.838818 ,  -9.474525 , -11.24898  ],
       [-13.95219  , -12.490049 , -17.123957 ],
       [-14.487818 , -11.3755455, -10.576363 ]], dtype=float32)>

追加のチェックとして、単一のバッチポイントの対数確率が以前の確率と一致することを確認します。

In [ ]:
(jds_ia.log_prob(shaped_sample)[3, 1] -
 jds_i.log_prob([shaped_sample[0][3, 1],
                 shaped_sample[1][3, 1],
                 shaped_sample[2][3, 1, :]]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

<a id="AutoBatching-For-The-Win"></a>

### 優れた自動バッチ処理


これで、すべてのデシデラタを処理する JointDistribution のバージョンができました。`log_prob` は、`tfd.Independent` の使用によりスカラーを返し、軸を追加してブロードキャストを修正したため、複数のサンプリングが機能するようになりました。

しかし、`JointDistributionSequentialAutoBatched` (JDSAB) と呼ばれるより簡単で優れた方法があります。

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

In [ ]:
jds_ab.log_prob(jds.sample())

<tf.Tensor: shape=(), dtype=float32, numpy=-12.954952>

In [ ]:
shaped_sample = jds_ab.sample([5, 3])
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.191533 , -10.43885  , -16.371655 ],
       [-13.292994 , -11.97949  , -16.788685 ],
       [-15.987699 , -13.435732 , -10.6029   ],
       [-10.184758 , -11.969714 , -14.275676 ],
       [-12.740775 , -11.5654125, -12.990162 ]], dtype=float32)>

In [ ]:
jds_ab.log_prob(shaped_sample) - jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

これはどのように機能するのでしょうか？深く理解するために[コード](https://github.com/tensorflow/probability/blob/main/tensorflow_probability/python/distributions/joint_distribution_auto_batched.py#L426)を読むこともできますが、ここでは、ほとんどのユースケースに十分な簡単な概要を提供します。

- 最初の問題は、`Y` の分布に`batch_shape=[7]` および `event_shape=[]` があったことで、`Independent` を使用して、バッチの次元をイベントの次元に変換しました。JDSAB は、要素の分布のバッチ形状を無視し、バッチ形状をモデルの全体的なプロパティとして扱い、`[]` と見なされます (`batch_ndims > 0` を設定して特に指定されていない限り)。結果は、上記で手動で行ったように、tfd.Independent を使用して要素の分布の*{nbsp}全*バッチ次元をイベント次元に変換するのと同じです。
- 2 番目の問題は、`m` と `b` の形状を変換して、複数のサンプルを作成するときに `X` で適切にブロードキャストできるようにする必要があることでした。JDSAB では、単一のサンプルを生成するモデルを記述し、TensorFlow の [vectorized_map](https://www.tensorflow.org/api_docs/python/tf/vectorized_map) を使用して、モデル全体を「リフト」して複数のサンプルを生成します。 (この機能は、JAX の [vmap](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#Auto-vectorization-with-vmap) に似ています。)

バッチ形状の問題をより詳細に調査するために、元のエラーのある同時分布 `jds`、バッチごとに修正された分布 `jds_i` と `jds_ia`、および自動バッチ処理された `jds_ab` のバッチ形状を比較します。

In [ ]:
jds.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([7])]

In [ ]:
jds_i.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ia.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ab.batch_shape

TensorShape([])

元の `jds` には、さまざまなバッチ形状の劣確率分布があることがわかります。`jds_i` と `jds_ia` では、同じ (空の) バッチ形状で劣確率分布を作成することにより、これを修正します。`jds_ab` には 1 つの (空の) バッチ形状があります。

`JointDistributionSequentialAutoBatched` はいくつかの追加の一般性を無料で提供しています。共変量 `X` (および暗黙的に観測値 `Y`) を 2 次元にするとします。

In [ ]:
X = np.arange(14).reshape((2, 7))
X

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13]])

`JointDistributionSequentialAutoBatched` は変更なしで機能します (`X` の形状は `jds_ab.log_prob` によってキャッシュされるため、モデルを再定義する必要があります)。

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

shaped_sample = jds_ab.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.1813647 , -0.85994506,  0.27593774],
        [-0.73323774,  1.1153806 ,  0.8841938 ],
        [ 0.5127983 , -0.29271227,  0.63733214],
        [ 0.2362284 , -0.919168  ,  1.6648189 ],
        [ 0.26317367,  0.73077047,  2.5395133 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.09636458,  2.0138032 , -0.5054413 ],
        [ 0.63941646, -1.0785882 , -0.6442188 ],
        [ 1.2310615 , -0.3293852 ,  0.77637213],
        [ 1.2115169 , -0.98906034, -0.07816773],
        [-1.1318136 ,  0.510014  ,  1.036522  ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 2, 7), dtype=float32, numpy=
 array([[[[-1.9685398e+00, -1.6832136e+00, -6.9127172e-01,
            8.5992378e-01, -5.3123581e-01,  3.1584005e+00,
            2.9044402e+00],
          [-2.5645006e-01,  3.1554163e-01,  3.1186538e+00,
            1.4272424e+00,  1.2843871e+00,  1.2266440e+00,
            1.2798605e+00]],
 
         [[ 1.5973477e+00,

In [ ]:
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-28.90071 , -23.052422, -19.851362],
       [-19.775568, -25.894997, -20.302256],
       [-21.10754 , -23.667885, -20.973007],
       [-19.249458, -20.87892 , -20.573763],
       [-22.351208, -25.457762, -24.648403]], dtype=float32)>

一方、慎重に作成された `JointDistributionSequential` は機能しなくなりました。

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

try:
  jds_ia.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3,1] vs. [2,7] [Op:Mul]


これを修正するには、`m` と `b` の両方に 2 番目の `tf.newaxis` を追加して、形状に一致させ、`Independent` の呼び出しで `reinterpreted_batch_ndims` を 2 に増やす必要があります。この場合、自動バッチ処理に形状の問題を処理させる方が手早く簡単で、より人間工学的です。

繰り返しますが、このノートブックでは `JointDistributionSequentialAutoBatched` を見てきましたが、`JointDistribution` の他のバリアントには同等の `AutoBatched` があることに注意してください。(`JointDistributionCoroutine` を使用する場合、`JointDistributionCoroutineAutoBatched` には、`Root` ノードを指定する必要がなくなるという追加の利点があります。`JointDistributionCoroutine` を使用したことがない場合は、この説明を無視しても問題ありません。）

### 最後に

このノートブックでは、`JointDistributionSequentialAutoBatched` を紹介し、簡単な例を詳しく説明しました。TFP の形状と自動バッチ処理について理解を深めてもらえたら幸いです。